In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('../')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, precision_recall_curve

import tensorflow as tf
import keras 

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, InputLayer, Flatten, Conv2D, MaxPooling2D, concatenate, Input, BatchNormalization, LocallyConnected2D

np.random.seed(42)
tf.random.set_seed(42)

from preprocessing.getdata import *

In [2]:
X_train0, X_test0, y_train0, y_test0 = get_tts(dim=0, N=5)
X_train1, X_test1, y_train1, y_test1 = get_tts(dim=1, N=5)
X_train2, X_test2, y_train2, y_test2 = get_tts(dim=2, N=5)

In [3]:
y_train = y_train0
y_test = y_test0

In [21]:
Input0 = Input(shape=X_train0[0].shape, name='input0')
Input1 = Input(shape=X_train1[0].shape, name='input1')
Input2 = Input(shape=X_train2[0].shape, name='input2')

b_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet')

for layer in b_model.layers:
    layer.trainable=False

x = concatenate([Flatten()(b_model(Input0)), Flatten()(b_model(Input1)), Flatten()(b_model(Input2))])
x = Dense(512, activation='relu')(x)
out = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[Input0, Input1, Input2], outputs=out)

In [8]:
Input0 = Input(shape=X_train0[0].shape, name='input0')
Input1 = Input(shape=X_train1[0].shape, name='input1')
Input2 = Input(shape=X_train2[0].shape, name='input2')

b_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet')

for layer in b_model.layers:
    layer.trainable=True

def build_model(Input):
  x = b_model(Input)
  x = BatchNormalization()(x)
  x = LocallyConnected2D(4, 3, 1)(x)
  x = Flatten()(x)
  return Dense(256, activation='relu', kernel_regularizer='l2')(x)

x = concatenate([build_model(Input0), build_model(Input1), build_model(Input2)])
#out = Average()(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu', kernel_regularizer='l2')(x)
out = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[Input0, Input1, Input2], outputs=out)

In [9]:
# Defining optimizer and learning rate
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.00001,
    decay_steps=10000,
    decay_rate=1,
    staircase=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, name='Adam')
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

training = model.fit([X_train0, X_train1, X_train2], y_train, epochs=30, shuffle=True, validation_data=([X_test0, X_test1, X_test2], y_test))

Epoch 1/30
157/157 [==============================] - 2944s 19s/step - loss: 8.8253 - accuracy: 0.6821 - val_loss: 8.6596 - val_accuracy: 0.6471
Epoch 2/30
 76/157 [=============>................] - ETA: 25:17 - loss: 8.3525 - accuracy: 0.8388

KeyboardInterrupt: 

In [ ]:
plt.plot(training.history['accuracy'])
plt.plot(training.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.ylim([0.5,1.])
plt.show()

NameError: name 'training' is not defined

In [1]:
y_predd = model.predict(X_test)
y_pred = (y_predd>0.5).astype(int)

# Plotting the confusing matrix
mat = confusion_matrix(y_test, y_pred.round())
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('true label')
plt.ylabel('predicted label')
print('accuracy: ' , accuracy_score(y_test, y_pred.round()).round(2))
print('recall: ' , recall_score(y_test, y_pred.round()).round(2))
print('precision: ' , precision_score(y_test, y_pred.round()).round(2))

NameError: name 'model' is not defined